In [1]:
import os
import pickle
import numpy as np
import cv2

#path_train = "C:\\Users\\me\\Jupiter_Notebooks\\Dataset\\Train"
#path_test = "C:\\Users\\me\\Jupiter_Notebooks\\Dataset\\Test"

In [2]:
#Import libraries and packages

import tensorflow as tf
from transformers import ViTFeatureExtractor, TFViTModel

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.layers import Conv2D, GlobalMaxPool2D

from keras.models import Sequential
from keras.layers.core import Dense, Activation

from keras.applications.vgg16 import VGG16 
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.layers import Input
from keras.models import Model

from tensorflow.keras.optimizers import Adam, SGD

In [4]:
os.getcwd()

'C:\\Users\\maria\\Jupiter_Notebooks\\Dataset_Thermal\\SeekThermal'

In [5]:
path_train = "../SeekThermal/Train"
path_test = "../SeekThermal/Test"

In [6]:
# defining the class for the train dataset

class MasterImage(object):

    def __init__(self,PATH='', IMAGE_SIZE = 300):
        self.PATH = PATH
        self.IMAGE_SIZE = IMAGE_SIZE

        self.image_data = []
        self.x_data = []
        self.y_data = []
        self.CATEGORIES = []

        # This will get List of categories
        self.list_categories = []

    def get_categories(self):
        for path in os.listdir(self.PATH):
            if '.DS_Store' in path:
                pass
            else:
                self.list_categories.append(path)
        print("Found Categories ",self.list_categories,'\n')
        return self.list_categories

    def process_image(self):
        try:
            """
            Return Numpy array of image
            :return: X_Data, Y_Data
            """
            self.CATEGORIES = self.get_categories()
            for categories in self.CATEGORIES:                                                  # Iterate over categories

                train_folder_path = os.path.join(self.PATH, categories)                         # Folder Path
                class_index = self.CATEGORIES.index(categories)                                 # this will get index for classification

                for img in os.listdir(train_folder_path):                                       # This will iterate in the Folder
                    new_path = os.path.join(train_folder_path, img)                             # image Path

                    try:        # if any image is corrupted
                        image_data_temp = cv2.imread(new_path)                 # Read Image as numbers
                        image_temp_resize = cv2.resize(image_data_temp,(self.IMAGE_SIZE,self.IMAGE_SIZE))
                        self.image_data.append([image_temp_resize,class_index])
                        random.shuffle(self.image_data)
                    except:
                        pass

            data = np.asanyarray(self.image_data,dtype=object)

            # Iterate over the Data
            for x in data:
                self.x_data.append(x[0])        # Get the X_Data
                self.y_data.append(x[1])        # get the label

            X_Data = np.asarray(self.x_data) / (255.0)      # Normalize Data
            Y_Data = np.asarray(self.y_data)

            # reshape x_Data

            X_Data = X_Data.reshape(-1, self.IMAGE_SIZE, self.IMAGE_SIZE, 3)

            return X_Data, Y_Data
        except:
            print("Failed to run Function Process Image ")

    def pickle_image(self):

        """
        :return: None Creates a Pickle Object of DataSet
        """
        # Call the Function and Get the Data
        X_Data,Y_Data = self.process_image()

        # Write the Entire Data into a Pickle File
        pickle_out = open('..\SeekThermal\X_Data','wb')
        pickle.dump(X_Data, pickle_out)
        pickle_out.close()

        # Write the Y Label Data
        pickle_out = open('..\SeekThermal\Y_Data', 'wb')
        pickle.dump(Y_Data, pickle_out)
        pickle_out.close()

        print("Pickled Image Successfully ")
        return X_Data,Y_Data

    def load_dataset(self):

        try:
            # Read the Data from Pickle Object
            X_Temp = open('..\SeekThermal\X_Data','rb')
            X_Data = pickle.load(X_Temp)

            Y_Temp = open('..\SeekThermal\Y_Data','rb')
            Y_Data = pickle.load(Y_Temp)

            print('Reading Dataset from Pickle Object')

            return X_Data,Y_Data

        except:
            print('Could not Found Pickle File ')
            print('Loading File and Dataset  ..........')

            X_Data,Y_Data = self.pickle_image()
            return X_Data,Y_Data

In [7]:
a = MasterImage(PATH = path_train,IMAGE_SIZE = 224)

In [8]:
array_a = a.load_dataset()
array_a

Reading Dataset from Pickle Object


(array([[[[0.97254902, 0.99607843, 1.        ],
          [0.98823529, 0.99607843, 1.        ],
          [0.99215686, 0.99607843, 0.99607843],
          ...,
          [0.70588235, 0.16862745, 0.        ],
          [0.73333333, 0.18823529, 0.00392157],
          [0.75686275, 0.21176471, 0.01568627]],
 
         [[1.        , 0.98823529, 0.99215686],
          [1.        , 0.98039216, 1.        ],
          [0.99607843, 0.98823529, 0.99607843],
          ...,
          [0.68627451, 0.17254902, 0.        ],
          [0.70980392, 0.18823529, 0.        ],
          [0.73333333, 0.2       , 0.        ]],
 
         [[0.99607843, 0.98431373, 0.99215686],
          [0.97647059, 0.98039216, 1.        ],
          [0.96078431, 0.98039216, 1.        ],
          ...,
          [0.70980392, 0.16862745, 0.        ],
          [0.73333333, 0.18823529, 0.00392157],
          [0.75686275, 0.20784314, 0.00784314]],
 
         ...,
 
         [[0.00392157, 0.        , 0.00784314],
          [0.01176

In [9]:
(train_images, train_labels) = array_a

In [10]:
print(train_images.shape, train_labels.shape)

(4732, 224, 224, 3) (4732,)


In [11]:
# defining the class for the test dataset

class MasterImage1(object):

    def __init__(self,PATH='', IMAGE_SIZE = 50):
        self.PATH = PATH
        self.IMAGE_SIZE = IMAGE_SIZE

        self.image_data = []
        self.x_data = []
        self.y_data = []
        self.CATEGORIES = []

        # This will get List of categories
        self.list_categories = []

    def get_categories(self):
        for path in os.listdir(self.PATH):
            if '.DS_Store' in path:
                pass
            else:
                self.list_categories.append(path)
        print("Found Categories ",self.list_categories,'\n')
        return self.list_categories

    def process_image(self):
        try:
            """
            Return Numpy array of image
            :return: X_Data, Y_Data
            """
            self.CATEGORIES = self.get_categories()
            for categories in self.CATEGORIES:                                                  # Iterate over categories

                train_folder_path = os.path.join(self.PATH, categories)                         # Folder Path
                class_index = self.CATEGORIES.index(categories)                                 # this will get index for classification

                for img in os.listdir(train_folder_path):                                       # This will iterate in the Folder
                    new_path = os.path.join(train_folder_path, img)                             # image Path

                    try:        # if any image is corrupted
                        image_data_temp = cv2.imread(new_path)                 # Read Image as numbers
                        image_temp_resize = cv2.resize(image_data_temp,(self.IMAGE_SIZE,self.IMAGE_SIZE))
                        self.image_data.append([image_temp_resize,class_index])
                        random.shuffle(self.image_data)
                    except:
                        pass

            data = np.asanyarray(self.image_data)

            # Iterate over the Data
            for x in data:
                self.x_data.append(x[0])        # Get the X_Data
                self.y_data.append(x[1])        # get the label

            X_Data = np.asarray(self.x_data) / (255.0)      # Normalize Data
            Y_Data = np.asarray(self.y_data)

            # reshape x_Data

            X_Data = X_Data.reshape(-1, self.IMAGE_SIZE, self.IMAGE_SIZE, 3)

            return X_Data, Y_Data
        except:
            print("Failed to run Function Process Image ")

    def pickle_image(self):

        """
        :return: None Creates a Pickle Object of DataSet
        """
        # Call the Function and Get the Data
        X_Data,Y_Data = self.process_image()

        # Write the Entire Data into a Pickle File
        pickle_out = open('..\SeekThermal\X1_Data','wb')
        pickle.dump(X_Data, pickle_out)
        pickle_out.close()

        # Write the Y Label Data
        pickle_out = open('..\SeekThermal\Y1_Data', 'wb')
        pickle.dump(Y_Data, pickle_out)
        pickle_out.close()

        print("Pickled Image Successfully ")
        return X_Data,Y_Data

    def load_dataset(self):

        try:
            # Read the Data from Pickle Object
            X_Temp = open('..\SeekThermal\X1_Data','rb')
            X_Data = pickle.load(X_Temp)

            Y_Temp = open('..\SeekThermal\Y1_Data','rb')
            Y_Data = pickle.load(Y_Temp)

            print('Reading Dataset from Pickle Object')

            return X_Data,Y_Data

        except:
            print('Could not Found Pickle File ')
            print('Loading File and Dataset  ..........')

            X_Data,Y_Data = self.pickle_image()
            return X_Data,Y_Data

In [12]:
b = MasterImage1(PATH = path_test, IMAGE_SIZE = 224)

In [13]:
array_b = b.load_dataset()
array_b

Reading Dataset from Pickle Object


(array([[[[0.97254902, 0.99607843, 1.        ],
          [0.98823529, 0.99607843, 1.        ],
          [0.99215686, 0.99607843, 0.99607843],
          ...,
          [0.70588235, 0.16862745, 0.        ],
          [0.73333333, 0.18823529, 0.00392157],
          [0.75686275, 0.21176471, 0.01568627]],
 
         [[1.        , 0.98823529, 0.99215686],
          [1.        , 0.98039216, 1.        ],
          [0.99607843, 0.98823529, 0.99607843],
          ...,
          [0.68627451, 0.17254902, 0.        ],
          [0.70980392, 0.18823529, 0.        ],
          [0.73333333, 0.2       , 0.        ]],
 
         [[0.99607843, 0.98431373, 0.99215686],
          [0.97647059, 0.98039216, 1.        ],
          [0.96078431, 0.98039216, 1.        ],
          ...,
          [0.70980392, 0.16862745, 0.        ],
          [0.73333333, 0.18823529, 0.00392157],
          [0.75686275, 0.20784314, 0.00784314]],
 
         ...,
 
         [[0.00392157, 0.        , 0.00784314],
          [0.01176

In [14]:
(test_images, test_labels) = array_b

In [15]:
print(train_images.shape, train_labels.shape)

(4732, 224, 224, 3) (4732,)


### Defining the Man images category as '1' vs  Car and Cat as '0'

In [16]:
### Defining Man as 1 vs the other categories
test_labels_binary0 = np.where(test_labels==1, 0, test_labels) 
test_labels_binary0

array([0, 0, 0, ..., 2, 2, 2])

In [17]:
test_labels_binary = np.where(test_labels==2, 1, test_labels_binary0)
np.unique(test_labels_binary)

array([0, 1])

In [18]:
### Man
train_labels_binary0 = np.where(train_labels==1, 0, train_labels) 
train_labels_binary0

array([0, 0, 0, ..., 2, 2, 2])

In [19]:
train_labels_binary = np.where(train_labels==2, 1, train_labels_binary0)
train_labels_binary 

array([0, 0, 0, ..., 1, 1, 1])

In [20]:
train_images.shape[1:]

(224, 224, 3)

In [ ]:
## Reshaping the arrays

In [85]:
test_labels_binary = test_labels_binary.reshape(test_labels_binary.shape[0],1)
train_labels_binary = train_labels_binary.reshape(train_labels_binary.shape[0],1)

In [86]:
print('Train: X_images=%s, y_labels=%s' % (train_images.shape, train_labels_binary.shape))
print('Test: X_images=%s, y_labels=%s' % (test_images.shape, test_labels_binary.shape))

Train: X_images=(4732, 224, 224, 3), y_labels=(4732, 1)
Test: X_images=(1067, 224, 224, 3), y_labels=(1067, 1)


In [141]:
# Setting 17% of the training set as a validation set

#validation set
X_val_images = train_images[-800:]
y_val_labels = train_labels_binary[-800:]

#training set
X_train_images = train_images[:-800]
y_train_labels = train_labels_binary[:-800]

In [124]:
X_val_images.shape

(800, 224, 224, 3)

In [125]:
X_train_images = np.reshape(X_train_images, (3, 224,224,3932))
X_val_images = np.reshape(X_val_images, (3, 224,224,800))

In [142]:
print('Train: X_train_images=%s, y_train_labels=%s' % (X_train_images.shape, y_train_labels.shape))
print('Validation: X_val_images=%s, y_val_labels=%s' % (X_val_images.shape, y_val_labels.shape))

Train: X_train_images=(3932, 224, 224, 3), y_train_labels=(3932, 1)
Validation: X_val_images=(800, 224, 224, 3), y_val_labels=(800, 1)


### Using Vision Transformer (ViT) pre-trained on ImageNet-21k

In [25]:
# Flipping and rotating images
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [89]:
# Apply data augmentation
inputs = keras.Input(shape = X_train_images.shape[1:])
x = data_augmentation(inputs)  

In [27]:
base_model = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


### The model

In [143]:
# Freeze base model
base_model.trainable = False
# Create new model
inputs = keras.Input(shape = (3, 224, 224), dtype='float32')
x = data_augmentation(inputs) 

x = base_model(x, training=False)[0]

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [144]:
# model
model_vit = tf.keras.Model(inputs, outputs)
model_vit.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

model_vit.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        [(None, 3, 224, 224)]     0         
_________________________________________________________________
sequential (Sequential)      (None, None, 224, None)   0         
_________________________________________________________________
tf_vi_t_model (TFViTModel)   TFBaseModelOutputWithPool 86389248  
_________________________________________________________________
dense_29 (Dense)             (None, 197, 1)            769       
Total params: 86,390,017
Trainable params: 769
Non-trainable params: 86,389,248
_________________________________________________________________


In [145]:
# Train the Vit model
vit_trained_model = model_vit.fit( X_train_images, y_train_labels, validation_data=(X_val_images, y_val_labels),verbose=2, epochs=200)

scores = model_xcept.evaluate(test_images, test_labels_binary, verbose=0)
print("Xception Accuracy on Test Set: %.2f%%" % (scores[1]*100))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.